In [1]:
import logic.pulsed.pulse_objects as po
from logic.pulsed.sampling_functions import SamplingFunctions as SF
import time
import matplotlib.pyplot as plt
pjl = pulsedjupyterlogic_AWG #Make sure the Logic is started in the Qudi manager!!!!

#### Current  philosophy of pulse sequence creation
1. For all the pulsed measurement that involve possible multiple phases and different pulse lengths - it is preferred and also implement right now - a long AWG sequence covering all taus is run with a single trigger from the PulseStreamer

2. For T1 sequence since it is very long and we have issues possibly due to memory - we individually trigger the AWG pi pulse before every needed pulse with the AWG sync time added in - this is also how the PODMR logic is implemented (albeit for many different freqs. so actually a different ensemble is being triggered unlike T1) - uses the sample_load_multi_short_sine_AWG function.

3. This is also valid for any other pulse sequence imaginable where individual triggering would be preferred - as long as the length of the triggerable ensembles are equal or only single ensemble is necessary.

4. Both cam be run in AWG switch mode = True (Zaswa is constantly on and AWG control the pulsing - signficantly better pulse shape and results compared to the other mode - highly recommended!).

5. Tracking mode for qAFM is also implemented such that the MW switch is always on and the AWG is controlling the on - off times

6. MW_0 and all _0 is associated with the primary (SMBV) MW source that will drive NV electron spin. MW_1 is for the secondary
SGS100A device.

#### Define general measurement params

In [4]:
#measurement parameters
laser_power_volatge = 0.08
pi_pulse = 69e-9 

#Driving information for the first LO (SMBV)
target_freq_0 = 2.15887e9
LO_freq_0 = target_freq_0 + 100e6
power_0 = -7


#Driving information for the second LO (SGS)
target_freq_1 = 2.15848e9
LO_freq_1 = target_freq_1 + 100e6
power_1 = -100#-20.7

#additional information for save tag
tip_name = 'L-R-07-24'
sample = 'NbSe2_S4_rot'
temperature = '2K'
b_field = '24mT_Bnv'
contact = 'OOC'
extra = 'test'
#x10.8µm_y4µm_Meissner_C0_SetA

#### Load and ready Rabi sequence

In [6]:
tau_start = 0
tau_stop = 200e-9
tau_num = 30

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.Rabi(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Rabi', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready PODMR sequence

In [9]:
mw_start = 2.14e9
mw_stop = 2.18e9
mw_step = 1e6

if LO_freq_0<mw_stop+mw_step:
    print('!!!Warning! LO Freq too low. Sweep is too large in range!!!\n\n\n')
pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.PODMR(mw_start, mw_stop, mw_step)
pjl.start_measurement(measurement_type='PODMR', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Tracking mode sequence

In [14]:
res_freq = target_freq_0
delta_freq = 5e6

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.Tracking(res_freq, delta_freq)
pjl.start_measurement(measurement_type='Tracking', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready optical T1 sequence

In [2]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.T1_optical_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-opt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready T1 sequence (alternating)

In [11]:
tau_start = 1e-6
tau_stop = 0.5e-3
tau_num = 10

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.T1_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54002 GHz
Using target_freq:  2.44002 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.99473660e-06 3.97897410e-06 7.93700526e-06
 1.58322349e-05 3.15811383e-05 6.29960525e-05 1.25660531e-04
 2.50659661e-04 5.00000000e-04]
number_of_lasers:
  20
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-alt_A-T17-12_1.9K_200mT_Bnv_OOC_AWGSWitch-mode_pi_pulse_calc


#### Load and ready T1 sequence init dark (alternating)

In [3]:
tau_start = 1e-6
tau_stop = 500e-3
tau_num = 5

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.T1_dark_init_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-dark-init-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready T1 SQ sequence (alternating)

In [33]:
tau_start = 1e-6
tau_stop = 40e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.T1_SQ_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-SQ-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.08 V
Using LO_freq:  2.54057 GHz
Using target_freq:  2.44057 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.74668135e-06 3.05089575e-06 5.32894271e-06
 9.30796486e-06 1.62580486e-05 2.83976304e-05 4.96016115e-05
 8.66382098e-05 1.51329346e-04 2.64324146e-04 4.61690057e-04
 8.06425413e-04 1.40856823e-03 2.46031986e-03 4.29739482e-03
 7.50617940e-03 1.31109036e-02 2.29005708e-02 4.00000000e-02]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-Pi']
Save_tag:  T1-SQ-alt_A-T17-12_1.9K_200mT_Bnv_OOC_Set_A


#### Load and ready T1 DQ sequence (alternating)

In [13]:
tau_start = 1e-6
tau_stop = 1e-3
tau_num = 20

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.T1_DQ_alt_exp(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='T1-DQ-alt', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.07 V
Using LO_freq:  2.881845 GHz
Using target_freq:  2.78469 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.00000000e-06 1.43844989e-06 2.06913808e-06 2.97635144e-06
 4.28133240e-06 6.15848211e-06 8.85866790e-06 1.27427499e-05
 1.83298071e-05 2.63665090e-05 3.79269019e-05 5.45559478e-05
 7.84759970e-05 1.12883789e-04 1.62377674e-04 2.33572147e-04
 3.35981829e-04 4.83293024e-04 6.95192796e-04 1.00000000e-03]
number_of_lasers:
  40
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
Save_tag:  T1-DQ-alt_A-T17-12_1.9K_0mT_Bnv_OOC


#### Load and ready Ramsey sequence with phase shift (alternating)

In [39]:
tau_start = 0
tau_stop = 10e-6
tau_num = 200

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.Ramsey_alt_phased(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Ramsey-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready Hahn Echo sequence with phase shift (alternating)

In [7]:
tau_start = 200e-9
tau_stop = 10e-6
tau_num = 200

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.Hecho_alt_phased(tau_start, tau_stop, tau_num)
pjl.start_measurement(measurement_type='Hecho-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready DEER sequence with phase shift (alternating)

In [39]:
mw_start = 5.3e9
mw_stop = 5.64e9
mw_step = 1e6
tau_NV = 2e-6
pi_pulse_dark_spin = tau_NV/2

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.DEER_alt_phased(tau_NV=tau_NV, pi_pulse_dark_spin=pi_pulse_dark_spin, mw_start=mw_start, mw_stop=mw_stop, mw_step=mw_step)
pjl.start_measurement(measurement_type='DEER-alt-phased', tip_name=tip_name, sample=sample, 
                      temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.0615 V
invoke_settings:
  False
controlled_variable:
  [5.300e+09 5.301e+09 5.302e+09 5.303e+09 5.304e+09 5.305e+09 5.306e+09
 5.307e+09 5.308e+09 5.309e+09 5.310e+09 5.311e+09 5.312e+09 5.313e+09
 5.314e+09 5.315e+09 5.316e+09 5.317e+09 5.318e+09 5.319e+09 5.320e+09
 5.321e+09 5.322e+09 5.323e+09 5.324e+09 5.325e+09 5.326e+09 5.327e+09
 5.328e+09 5.329e+09 5.330e+09 5.331e+09 5.332e+09 5.333e+09 5.334e+09
 5.335e+09 5.336e+09 5.337e+09 5.338e+09 5.339e+09 5.340e+09 5.341e+09
 5.342e+09 5.343e+09 5.344e+09 5.345e+09 5.346e+09 5.347e+09 5.348e+09
 5.349e+09 5.350e+09 5.351e+09 5.352e+09 5.353e+09 5.354e+09 5.355e+09
 5.356e+09 5.357e+09 5.358e+09 5.359e+09 5.360e+09 5.361e+09 5.362e+09
 5.363e+09 5.364e+09 5.365e+09 5.366e+09 5.367e+09 5.368e+09 5.369e+09
 5.370e+09 5.371e+09 5.372e+09 5.373e+09 5.374e+09 5.375e+09 5.376e+09
 5.377e+09 5.378e+09 5.379e+09 5.380e+09 5.381e+09 5.382e+09 5.383e+09
 5.384e+09 5.385e+09 5.386e+09 5.387e+09 5.388e+09 5.389e+09 5.390e+0

#### Load and ready DEER CPMG sequence with phase shift (alternating)

In [10]:
mw_start = 5.3e9
mw_stop = 5.62e9
mw_step = 1e6
tau_NV = 10e-6
N = 16
pi_pulse_dark_spin = (tau_NV- 2*N*pi_pulse)/(2*N)

pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
pjl.DEER_CPMG_alt_phased(tau_NV=tau_NV, pi_pulse_dark_spin=pi_pulse_dark_spin, mw_start=mw_start, mw_stop=mw_stop, mw_step=mw_step, N=N)
pjl.start_measurement(measurement_type='DEER-CPMG-alt-phased', tip_name=tip_name, sample=sample, 
                        temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready CPMG sequence (alternating)

In [38]:
tau_start = 8e-6
tau_stop = 12e-6
tau_num = 10
N = 16

if tau_start < 2*N*pi_pulse:
    min_tau = pi_pulse*2*N*1e9 #minimum tau in ns
    print(f'!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.CPMG_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'CPMG({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.0615 V
invoke_settings:
  False
controlled_variable:
  [8.00000000e-06 8.44444444e-06 8.88888889e-06 9.33333333e-06
 9.77777778e-06 1.02222222e-05 1.06666667e-05 1.11111111e-05
 1.15555556e-05 1.20000000e-05]
number_of_lasers:
  20
laser_ignore_list:
  []
alternating:
  True
units:
  ('s', 'arb. u.')
labels:
  ['Tau', 'Signal']
['Jupyter-ensemble-cpmg-alt-phased-juptr']
Save_tag:  CPMG(16)-alt-phased_A-T17-12_NbSe2_S2_7.0K_200mT_Bnv_OOC_test


#### Load and ready XY4 sequence (alternating)

In [ ]:
tau_start = 18e-6
tau_stop = 36e-6
tau_num = 200
N = 8

if tau_start/(4*N) < pi_pulse:
    min_tau = pi_pulse*4*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY4_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY4({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [ ]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*4
tau_stop = pi_pulse_spacing_stop*N*4
tau_num = pi_pulse_spacing_num


if tau_start/(4*N) < pi_pulse:
    min_tau = pi_pulse*4*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY4_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY4({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready XY8 sequence (alternating)

In [26]:
tau_start = 18e-6
tau_stop = 36e-6
tau_num = 200
N = 8

if tau_start/(8*N) < pi_pulse:
    min_tau = pi_pulse*8*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY8_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY8({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

Using laser voltage:  0.059 V
Using LO_freq:  2.53598 GHz
Using target_freq:  2.43598 GHz
Using power:  -1 dBm
invoke_settings:
  False
controlled_variable:
  [1.80000000e-05 1.80904523e-05 1.81809045e-05 1.82713568e-05
 1.83618090e-05 1.84522613e-05 1.85427136e-05 1.86331658e-05
 1.87236181e-05 1.88140704e-05 1.89045226e-05 1.89949749e-05
 1.90854271e-05 1.91758794e-05 1.92663317e-05 1.93567839e-05
 1.94472362e-05 1.95376884e-05 1.96281407e-05 1.97185930e-05
 1.98090452e-05 1.98994975e-05 1.99899497e-05 2.00804020e-05
 2.01708543e-05 2.02613065e-05 2.03517588e-05 2.04422111e-05
 2.05326633e-05 2.06231156e-05 2.07135678e-05 2.08040201e-05
 2.08944724e-05 2.09849246e-05 2.10753769e-05 2.11658291e-05
 2.12562814e-05 2.13467337e-05 2.14371859e-05 2.15276382e-05
 2.16180905e-05 2.17085427e-05 2.17989950e-05 2.18894472e-05
 2.19798995e-05 2.20703518e-05 2.21608040e-05 2.22512563e-05
 2.23417085e-05 2.24321608e-05 2.25226131e-05 2.26130653e-05
 2.27035176e-05 2.27939698e-05 2.28844221e-05 2.

In [ ]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*8
tau_stop = pi_pulse_spacing_stop*N*8
tau_num = pi_pulse_spacing_num


if tau_start/(8*N) < pi_pulse:
    min_tau = pi_pulse*8*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY8_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY8({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

#### Load and ready XY16 sequence (alternating)

In [ ]:
tau_start = 18e-6
tau_stop = 36e-6
tau_num = 200
N = 8

if tau_start/(16*N) < pi_pulse:
    min_tau = pi_pulse*16*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY16_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY16({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [143]:
pi_pulse_spacing_start = 100e-9
pi_pulse_spacing_stop = 600e-9
pi_pulse_spacing_num = 100
N = 8

tau_start = pi_pulse_spacing_start*N*16
tau_stop = pi_pulse_spacing_stop*N*16
tau_num = pi_pulse_spacing_num


if tau_start/(16*N) < pi_pulse:
    min_tau = pi_pulse*16*N*1e9 #minimum tau in ns
    print('!!!Warning. tau_start value is too small!!!\n It has to be larger than {min_tau} ns!')
else:
    pjl.initialize_ensemble(laser_power_volatge = laser_power_volatge, pi_pulse=pi_pulse, LO_freq_0=LO_freq_0, target_freq_0=target_freq_0, power_0=power_0, LO_freq_1=LO_freq_1, target_freq_1=target_freq_1, power_1=power_1)
    pjl.XY16_alt_phased(tau_start, tau_stop, tau_num, N)
    pjl.start_measurement(measurement_type=f'XY16({N})-alt-phased', tip_name=tip_name, sample=sample, 
                          temperature=temperature, b_field=b_field, contact=contact, extra=extra)

In [6]:
afm_scanner_logic._mw.off()
afm_scanner_logic._mw1.off()

0